In [257]:
import pandas as pd
import matplotlib as plt
import re
%matplotlib inline
dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/'

## make master list of utility names from utility dataset, generator dataset, and owners dataset

### Read in relevant datasets

In [15]:
owners = pd.read_csv(dataDir + 'master_company_list_2018/owners2018.csv')
gen_data = pd.read_csv(dataDir + 'master_company_list_2018/gen_2018.csv')
utils = pd.read_csv(dataDir + 'master_company_list_2018/utils_2018.csv')

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
dfList = [owners, gen_data, utils]


In [22]:
owners.columns

Index(['Unnamed: 0', 'utility_id', 'utility_name', 'plant_code', 'plant_name',
       'state', 'generator_id', 'status', 'owner_name', 'owner_street_address',
       'owner_city', 'owner_state', 'owner_zip', 'ownership_id',
       'percent_owned', 'perc_owned', 'num_owners', 'idx', 'owner_1',
       'owner_2', 'owner_3', 'owner_4', 'perc_owner_1', 'perc_owner_2',
       'perc_owner_3', 'perc_owner_4'],
      dtype='object')

In [28]:
owners.rename(columns = {'owner_name':'company_name', 
                         'owner_street_address':'company_address',
                         'owner_state': 'company_state',
                          'owner_city': 'company_city',
                          'owner_zip': 'company_zip',
                           'ownership_id': 'company_id'},inplace=True)
gen_data.rename(columns = {'utility_name':'company_name'}, inplace=True)
utils.rename(columns = {'utility_name':'company_name',
                       'utility_id': 'company_id',
                        'street_address':'company_address',
                         'state':'company_state',
                         'city': 'company_city', 
                          'zip':'company_zip'},inplace=True)

In [299]:
# make df with master list of companies (utilities + owners)
companyVals = ['name', 'address','state','city','zip','id']
companyVals = ['company_' + x for x in companyVals]
companies = owners[companyVals].append(utils[companyVals])
# select unique combos
companies.drop_duplicates(inplace=True)

In [280]:
names = gen_data['company_name'].drop_duplicates()
# I found that no names in gen data are not contained in the other datasets, so I don't merge it in

In [281]:
def shorten_rd(address):
    '''Completes the road type. I.e. Rd becomes Road, st becomes Street as per Google etc.'''
    address = address.title()
    address = re.sub(r" Street(?=$| [NE(So|S$)(We|W$)])", ' St', address)
    address = re.sub(r" Road(?=$| [NE(So|S$)(We|W$)])", ' Rd', address)
    address = re.sub(r"(?<!The) Avenue(?=$| [NE(So|S$)(We|W$)])", ' Ave', address)
    address = re.sub(r" Close(?=$| [NE(So|S$)(We|W$)])", ' Cl', address)
    address = re.sub(r" Court(?=$| [NE(So|S$)(We|W$)])", ' Ct', address)
    address = re.sub(r"(?<!The) Crescent(?=$| [NE(So|S$)(We|W$)])", ' Cres', address)
    address = re.sub(r" Boulevarde?(?=$| [NE(So|S$)(We|W$)])", ' Blvd', address)
    address = re.sub(r" Drive(?=$| [NE(So|S$)(We|W$)])", ' Dr', address)
    address = re.sub(r" Lane(?=$| [NE(So|S$)(We|W$)])", ' Ln', address)
    address = re.sub(r" Place(?=$| [NE(So|S$)(We|W$)])", ' Pl', address)
    address = re.sub(r" Square(?=$| [NE(So|S$)(We|W$)])", ' Sq', address)
    address = re.sub(r"(?<!The) Parade(?=$| [NE(So|S$)(We|W$)])", ' Pde', address)
    address = re.sub(r" Circuit(?=$| [NE(So|S$)(We|W$)])", ' Cct', address)
    return address

def lengthen_rd(address):
    address = address.title()
    address = re.sub(r" St(?=$| [NE(So|S$)(We|W$)])", " Street", address)
    address = re.sub(r" Rd(?=$| [NE(So|S$)(We|W$)])", " Road", address)
    address = re.sub(r" Ave(?=$| [NE(So|S$)(We|W$)])", " Avenue", address)
    address = re.sub(r" Cl(?=$| [NE(So|S$)(We|W$)])", " Close", address)
    address = re.sub(r" Ct(?=$| [NE(So|S$)(We|W$)])", " Court", address)
    address = re.sub(r" Cres(?=$| [NE(So|S$)(We|W$)])", " Crescent", address)
    address = re.sub(r" Blvd(?=$| [NE(So|S$)(We|W$)])", " Boulevard", address)
    address = re.sub(r" Dr(?=$| [NE(So|S$)(We|W$)])", " Drive", address)
    address = re.sub(r" Ln(?=$| [NE(So|S$)(We|W$)])", " Lane", address)
    address = re.sub(r" Pl(?=$| [NE(So|S$)(We|W$)])", " Place", address)
    address = re.sub(r" Sq(?=$| [NE(So|S$)(We|W$)])", " Square", address)
    address = re.sub(r" Pde(?=$| [NE(So|S$)(We|W$)])", " Parade", address)
    address = re.sub(r" Cct(?=$| [NE(So|S$)(We|W$)])", " Circuit", address)
    return address

def standard_addr(address):
    '''Checks for unit numbers and street addresses and puts them in the standard format''' 
    #print("################################")
    #print("### Address: ", address)
    unit_nums = re.findall(r"(?<=Unit )\w?\d+\w?|(?<=U)\d+\w?|\w?\d+\w?(?=\s*/)", address)
    unit_num = unit_nums[0] if len(unit_nums)==1 else ""
    #print("Unit Number: ", unit_num)
    proc_addr = re.sub(r"Unit \w?\d+\w?/?|U\d+\w?/?|\w?\d+\w?\s*/", "", address)
    proc_addr = re.sub(r"^[,\- ]+|[,\- ]+$", "", proc_addr)
    #print("Unitless address: ", proc_addr)
    type_opts = r"Terrace|Way|Walk|St|Rd|Ave|Cl|Ct|Cres|Blvd|Dr|Ln|Pl|Sq|Pde|Cct"
    road_attrs_pattern = r"(?P<rd_no>\w?\d+(\-\d+)?\w?\s+)(?P<rd_nm>[a-zA-z \d\-]+)\s+(?P<rd_tp>" + type_opts + ")"
    #print("Road Attr Pattern: ", road_attrs_pattern)
    road_attrs = re.search(road_attrs_pattern, proc_addr)
    try:
        road_num = road_attrs.group('rd_no').strip()
    except AttributeError:
        road_num = ""
    #print("Road number: ", road_num)
    try:
        road_name = road_attrs.group('rd_nm').strip()
    except AttributeError:
        road_name = ""
    #print("Road name: ", road_name)
    try:
        road_type = road_attrs.group('rd_tp').strip()
    except AttributeError:
        road_type = ""
    #print("Road type: ", road_type)
    proc_addr = lengthen_rd(re.sub(r"^[,\- ]+|[,\- ]+$", "", re.sub(road_attrs_pattern, "", proc_addr)))
    #print("Leftover: ", proc_addr)

    unit_seg = (unit_num + "/" if unit_num!="" else "") if road_num != "" else ("Unit " + unit_num + ", " if unit_num!="" else "")
    road_seg = ((road_num + " " if road_num!="" else "") + road_name + " " + road_type).strip()
    post_road_seg = " " + proc_addr if proc_addr != "" else ""
    proc_addr = (unit_seg + road_seg) + post_road_seg
    #print("### Processed Address: ", proc_addr)
    return proc_addr

In [319]:
# clean addresses
companies['raw_address'] = companies['company_address']
companies['company_address'] =companies['raw_address'].apply(lambda x: standard_addr(str(x)))

In [327]:
# first assign company name a common var
companies['name_group'] = companies.groupby('company_name').grouper.group_info[0]
# drop non-unique rows
temp = companies.drop_duplicates(subset='company_name', keep="first")

# now assign clean address common var
temp['address_group']=temp.groupby('company_address').grouper.group_info[0]
temp = temp.drop_duplicates(subset='company_address', keep="first")

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [331]:
# reduced number to 4000
# now do parent dict... can drop those and deal with remainder

,company_name,company_address,company_state,company_city,company_zip,company_id,address_group,name_group,raw_address
498,126 Grove Solar LLC,315 Post Road West,CT,Westport,6880,60517,640,3,315 Post Road West
3673,158th Fighter Wing,105 NCO Dr Ive,VT,South Burlington,05403,60301,2005,4,105 NCO Drive
4453,"1634 Solar, LLC",1519 King St Reet,SC,Charleston,29405,61541,2407,5,1519 King Street
4397,174 Power Global Corp.,300 Spectrum Center Dr . Ste1020,CA,Irvine,92618,61222,3017,6,300 Spectrum Center Dr. Ste1020
2137,"180 Raritan Energy Solutions, LLC","633 Division St Reet, Suite 2",NJ,Elizabeth,7201,58164,3665,7,"633 Division Street, Suite 2"
...,...,...,...,...,...,...,...,...,...
1519,Ziegler Power Systems,8050 State Highway 101East,MN,Shakopee,55379,21144,1106,6090,8050 State Highway 101East
1522,Zion Energy LLC,5701 Ninth St,IL,Zion,60099,21191,3566,6091,5701 Ninth St
2371,Zotos International,300 Forge Ave Nue,NY,Geneva,14456,56977,3006,6092,300 Forge Avenue
3914,eBay - South Jordan,6614 West Crimson View Dr,UT,South Jordan,84095,59095,3711,6094,6614 West Crimson View Dr


In [282]:
# need to clean addresses

# clean_addresses.drop_duplicates().reset_index().sort_values(by='company_address')

0        165
1       3565
5       2454
14      1448
15      3905
        ... 
5034    4456
5035    3031
5036    2924
5037    3977
5038    3978
Name: name_group, Length: 6197, dtype: int64

In [306]:
# companies with the same address should have same company group id
companies['address_group'] = companies.groupby('company_address').grouper.group_info[0]


temp.drop_duplicates(subset='company_address',keep='first')

,company_name,company_address,company_state,company_city,company_zip,company_id,address_group
0,Alabama Power Co,P O Box 2641600 North 18th Street,AL,Birmingham,35291,195,3668
1,Mississippi Power Co,2992 West Beach Boulevard,MS,Gulfport,39501,12686,1689
5,Georgia Power Co,P O Box 4545,GA,Atlanta,30302,7140,3746
14,Cleco Power LLC,2030 Donahue Ferry RoadP.O. Box 5000,LA,Pineville,71361,3265,1267
15,"Northeast Texas Elec Coop, Inc",1127 Judson Road,TX,Longview,75601,13670,374
...,...,...,...,...,...,...,...
5029,CA ODEVI LLC,"755 Baywood drive, 2nd Floor,",CA,Petaluma,94954,62772,2992
5031,Solar Hagerstown LLC,1260 Security Road,MD,Hagerstown,21742,62774,555
5033,Burrstone Energy Center,1656 Champlain Avenue,NY,New Hartford,13413,62777,950
5035,Kenyon Energy KS Solar 1 LLC,"11550 Ash St, Ste 300",KS,Leawood,66211,62784,412


In [291]:
companies['address_group_size'] = companies.groupby('address_group')['address_group'].transform('count')
companies.sort_values('address_group_size',ascending=False)
companies['city_id']=companies.groupby(['company_state','company_city']).grouper.group_info[0]
companies['city_group_size'] = companies.groupby('city_id')['city_id'].transform('count')
companies[companies['city_group_size'] > 1].sort_values('company_name')

,company_name,company_address,company_state,company_city,company_zip,company_id,raw_address,address_group,address_group_size,city_id,city_group_size
4964,"10 Briggs Solar NG, LLC","267 Water St Reet, 2Nd Floor",RI,Warren,02885,62685,"267 Water Street, 2nd Floor",2984,2,1906,2
4899,"1025 Traveller Solar, LLC",1447 S Tryon St Ste 201,NC,Charlotte,28203,62139,"1447 S Tryon St, Ste 201",2387,4,1318,154
4900,"1047 Little Mountain Solar, LLC",1447 S Tryon St Ste 201,NC,Charlotte,28203,62140,"1447 S Tryon St, Ste 201",2387,4,1318,154
3816,126 Grove Solar LLC,315 Post Road West,CT,Westport,06880,60517,315 Post Road West,647,9,412,21
498,126 Grove Solar LLC,315 Post Road West,CT,Westport,6880,60517,315 Post Road West,647,9,412,21
...,...,...,...,...,...,...,...,...,...,...,...
4217,Zumbro Solar LLC,C/O Amp Solar Group Inc. 55A P,ON,Port Credit,L5G 4,61172,c/o Amp Solar Group Inc. 55A P,1249,5,1768,5
3914,eBay - South Jordan,6614 West Crimson View Dr,UT,South Jordan,84095,59095,6614 West Crimson View Dr,3759,1,2093,3
3274,enXco Development Corporation,15445 Innovation Dr Ive,CA,San Diego,92128,56714,15445 Innovation Drive,2465,13,264,52
4230,esVolta LP,65 Enterprise,CA,Aliso Viejo,92656,61198,65 Enterprise,1018,1,97,5
